In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
# import de données  : 
df = pd.read_csv("/workspaces/credit-risk-/exemple.csv")


In [3]:

df.columns


Index(['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_status', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length'],
      dtype='object')

In [4]:

df.sample(10)


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
11252,22,53500,RENT,2.0,EDUCATION,B,10000,9.99,0,0.19,N,3
4887,23,38004,RENT,5.0,VENTURE,A,5000,6.91,0,0.13,N,2
17599,24,30000,RENT,6.0,DEBTCONSOLIDATION,A,6000,7.51,0,0.20,N,3
26688,29,110000,MORTGAGE,11.0,VENTURE,B,12000,10.37,0,0.11,N,10
1886,21,30000,MORTGAGE,5.0,PERSONAL,A,7600,6.03,0,0.25,N,4
7348,22,59000,RENT,2.0,EDUCATION,B,6500,10.95,0,0.11,N,3
28747,27,24000,RENT,0.0,VENTURE,B,2800,10.95,0,0.12,N,10
26136,31,84000,MORTGAGE,3.0,HOMEIMPROVEMENT,G,25000,17.34,1,0.30,Y,5
18277,29,46000,RENT,1.0,DEBTCONSOLIDATION,D,22750,14.46,1,0.49,N,9
27953,33,66000,OWN,16.0,HOMEIMPROVEMENT,A,3500,NaN,0,0.05,N,5


In [5]:
df.describe()


,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_cred_hist_length
count,32581.000000,3.258100e+04,31686.000000,32581.000000,29465.000000,32581.000000,32581.000000,32581.000000
mean,27.734600,6.607485e+04,4.789686,9589.371106,11.011695,0.218164,0.170203,5.804211
std,6.348078,6.198312e+04,4.142630,6322.086646,3.240459,0.413006,0.106782,4.055001
min,20.000000,4.000000e+03,0.000000,500.000000,5.420000,0.000000,0.000000,2.000000
25%,23.000000,3.850000e+04,2.000000,5000.000000,7.900000,0.000000,0.090000,3.000000
50%,26.000000,5.500000e+04,4.000000,8000.000000,10.990000,0.000000,0.150000,4.000000
75%,30.000000,7.920000e+04,7.000000,12200.000000,13.470000,0.000000,0.230000,8.000000
max,144.000000,6.000000e+06,123.000000,35000.000000,23.220000,1.000000,0.830000,30.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32581 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  32581 non-null  int64  
 1   person_income               32581 non-null  int64  
 2   person_home_ownership       32581 non-null  object 
 3   person_emp_length           31686 non-null  float64
 4   loan_intent                 32581 non-null  object 
 5   loan_grade                  32581 non-null  object 
 6   loan_amnt                   32581 non-null  int64  
 7   loan_int_rate               29465 non-null  float64
 8   loan_status                 32581 non-null  int64  
 9   loan_percent_income         32581 non-null  float64
 10  cb_person_default_on_file   32581 non-null  object 
 11  cb_person_cred_hist_length  32581 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.0+ MB
